In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from niklib.dataset_matify import MattextDataset
import torch
torch.cuda.set_device(0)
# mattext = MattextDataset.from_matifynet()
# mattext.save("./niklib/train.pickle")
mattext = MattextDataset.from_pickle("./niklib/train.pickle")

In [3]:
from niklib.w2v_model import Word2VecModel
#w2v = Word2VecModel.from_fasttext_vec('../wiki.no.vec')
#w2v.save('../wiki.no.pickle')
w2v = Word2VecModel.from_pickle('wiki.no.pickle')
w2v

Number of vocabularies: 515789 
Number of embedding factors: 300 

In [4]:
from niklib.dataset_text import TextDataset, TextModelData
from niklib.processor_text import TextProcessor

textproc = TextProcessor(w2v, max_pad = 50)
ds = TextDataset.from_mattext(mattext, textproc)
ds.save('./niklib/mattext_ds.pk')
modeldata = TextModelData.from_ds('Matify_data', ds, val_p = 0.2, test_p = 0.0, bs = 128)
modeldata

Text length: min=1, max=139, mean=10.538635572572224
Padding all text to have fixed length = 50


Data path: Matify_data 
Fixed sequence length: 50 
Number of classes: 51 
Training examples: 4596 
Validation examples: 1150 
Test examples: 0 

In [13]:
#TEXT = data.Field(lower=True, tokenize=spacy_tok)
#self.tokenize_fn = TEXT.preprocess

texts_test=['The burger Cheesy, original eller spicy. 4 x 80 g. 320 g. Pr kg 155,94', 
            'Coop svenske kjøttboller 400 g. Pr kg 69,00',
            'Barilla spaghetti 1 kg',
            'Barilla tagliatelle M/egg. 500 g. Pr kg 77,00',
            'Lofoten hjemmelaget fiskegrateng 400 g. Pr kg 195,50 600 Pr',
            'Barilla spaghetti fullkorn 1 kg',
            'Coop fra havet laksefilet 500 g. Pr kg 159,80',
            'Barilla saus classico 400 g. Pr kg 59,75',
            'Coop fra havet lettsaltet torskefilet M/skinn. 360 g. Pr kg 147,50',
            'Barilla pesto genovese 190 g. Pr kg 157,37']
#tokenize('miniplommetomat')
#tokenize('babygulrøtter')
troublesome_words = 'Snålegg kryddermix kjøttdeig'
textproc.tokenize(troublesome_words)


['snål', 'egg', 'krydder', 'mix', 'kjøttdeig']

In [6]:
import sys
from falib.learner import Learner
from falib.model import fit
from falib.core import *
from falib.rnn_reg import EmbeddingDropout, WeightDrop
from falib.metrics import accuracy_thresh, accuracy
from torch import nn, optim
import torch
from torchtext.datasets import language_modeling
import dill as pickle
# Download spacy package if does not have
#!python -m spacy download en

In [7]:
from falib.learner import Learner
from niklib.model_text import MaxRNN, AttentionBiRNN, MaxEmbedding, PoolingRNN, CNNRNN
from falib.model import model_summary # model_summary is not working with embedding layer
from falib.core import V
model = MaxRNN(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(),
                    classifier_szs = [1000], classifier_drops = [0.5],
                    hidden_size=600, n_layers=1, bidirectional=True, # 1000, 1
                    drop_emb=0.02, drop_in=0.05, drop_rnnw=0.1, 
                    drop_rnnh=0.05)
#model = PoolingRNN(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(),
#                    classifier_szs = [1000], classifier_drops = [0.05],
#                    hidden_size=300, n_layers=2, bidirectional=True,
#                    drop_emb=0.02, drop_in=0.05, drop_rnnw=0.1, 
#                    drop_rnnh=0.05)
#model = CNNRNN(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(),
#                 hidden_size=500, n_layers=2, bidirectional=True, 
#                 drop_emb=0.02, drop_in=0.05, drop_rnnw=0.1, drop_rnnh=0.05,
#                 kernel_szs=[3,4,5], 
#                 classifier_szs=[1000], classifier_drops=[0.5])

# model = MaxEmbedding(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(), drop_emb=0.02, drop_in=0.05, drop_classifier=[0.2, 0.2, 0.2])
# model = AttentionBiRNN(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(),
#                 hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
#model.load_state_dict(torch.load('niklib/MaxBiRNN0.91.pt'))
model = model.cuda()

In [8]:
# TODO: Set learning rate of embedding layer to be much lower than the rest
# TODO: Check models of the competition winner:
#       https://github.com/sermakarevich/jigsaw-toxic-comment-classification-challenge
# Recommended stuff:
#       http://forums.fast.ai/t/kaggle-nlp-competition-toxic-comment-classification-challenge/9055/41?u=dang.hien
# TODO: Learn about scikit pipeline:
#       https://www.kaggle.com/sermakarevich/sklearn-pipelines-tutorial/notebook
# TODO: Investigate model using lemme
#       https://github.com/marcotcr/lime
from falib.core import V
from functools import partial
#params = [param for name, param in model.named_parameters() if not name.startswith('wordemb')]
model.wordemb.emb.weight.requires_grad = True
learning_rate = 5e-5#Important #3e-3
#rnn_adam = partial(optim.Adam, betas=(0.7, 0.99))
#optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.7, 0.99), weight_decay=0)
# optimizer = torch.optim.Adam(params, lr=learning_rate)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate) # Work with sparse embedding, much faster, but worse result
loss_fn = torch.nn.NLLLoss()
i = 0
max_acc = 0.905
while i < 1000:
    model.train()
    train_acc = []
    for xs, ys in iter(modeldata.trn_dl):
        x_btch, y_btch = V(xs), V(ys)
        y_pred = model(x_btch)
        # Compute and print loss.
        loss = loss_fn(y_pred, y_btch)
        # train acc
        train_acc.append(accuracy(y_pred, y_btch))
        optimizer.zero_grad()
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
    
    model.eval()
    val_x = V(modeldata.val_ds.xs)
    val_y = modeldata.val_ds.ys
    val_y_pred = to_np(model(val_x)).argmax(axis=1)
    #model.training = True
    acc = np.sum(val_y_pred == val_y)/(len(val_y))
    if acc > max_acc:
        torch.save(model.state_dict(), 'MaxRNN'+str(round(acc,2))+'.pt')
        print('Saved')
        max_acc = acc
    print(f'Iter {i} - train loss: {to_np(loss)}, train acc: {to_np(sum(train_acc)/len(train_acc))}, val acc: {acc}')
    i = i + 1
    print()

/opt/conda/envs/fastai/lib/python3.6/site-packages/torch/nn/modules/module.py:325: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


Iter 0 - train loss: [-0.10239], train acc: [0.14576], val acc: 0.27304347826086955

Iter 1 - train loss: [-0.13515], train acc: [0.2721], val acc: 0.2808695652173913

Iter 2 - train loss: [-0.20117], train acc: [0.31138], val acc: 0.34956521739130436

Iter 3 - train loss: [-0.24206], train acc: [0.38772], val acc: 0.4260869565217391

Iter 4 - train loss: [-0.32577], train acc: [0.45402], val acc: 0.47304347826086957

Iter 5 - train loss: [-0.38622], train acc: [0.49554], val acc: 0.5173913043478261

Iter 6 - train loss: [-0.39442], train acc: [0.52991], val acc: 0.54

Iter 7 - train loss: [-0.52378], train acc: [0.55246], val acc: 0.5678260869565217

Iter 8 - train loss: [-0.47674], train acc: [0.57812], val acc: 0.5895652173913043

Iter 9 - train loss: [-0.49498], train acc: [0.59576], val acc: 0.6043478260869565

Iter 10 - train loss: [-0.5535], train acc: [0.60848], val acc: 0.6139130434782609

Iter 11 - train loss: [-0.58562], train acc: [0.62478], val acc: 0.6313043478260869

Ite

Iter 98 - train loss: [-0.90755], train acc: [0.92433], val acc: 0.88

Iter 99 - train loss: [-0.95755], train acc: [0.92589], val acc: 0.8791304347826087

Iter 100 - train loss: [-0.93942], train acc: [0.92634], val acc: 0.8826086956521739

Iter 101 - train loss: [-0.93144], train acc: [0.92567], val acc: 0.88

Iter 102 - train loss: [-0.9676], train acc: [0.92924], val acc: 0.8834782608695653

Iter 103 - train loss: [-0.93954], train acc: [0.92946], val acc: 0.8843478260869565

Iter 104 - train loss: [-0.95609], train acc: [0.93036], val acc: 0.8869565217391304

Iter 105 - train loss: [-0.9607], train acc: [0.92946], val acc: 0.8852173913043478

Iter 106 - train loss: [-0.90575], train acc: [0.93259], val acc: 0.8852173913043478

Iter 107 - train loss: [-0.91789], train acc: [0.93237], val acc: 0.8860869565217391

Iter 108 - train loss: [-0.93217], train acc: [0.93527], val acc: 0.8869565217391304

Iter 109 - train loss: [-0.95208], train acc: [0.93482], val acc: 0.8852173913043478



Saved
Iter 194 - train loss: [-0.98297], train acc: [0.96205], val acc: 0.9130434782608695

Iter 195 - train loss: [-0.95154], train acc: [0.96272], val acc: 0.908695652173913

Iter 196 - train loss: [-0.94268], train acc: [0.96317], val acc: 0.908695652173913

Iter 197 - train loss: [-0.96768], train acc: [0.9625], val acc: 0.9069565217391304

Iter 198 - train loss: [-0.94318], train acc: [0.96272], val acc: 0.9078260869565218

Iter 199 - train loss: [-0.9451], train acc: [0.96339], val acc: 0.9060869565217391

Iter 200 - train loss: [-0.95395], train acc: [0.96317], val acc: 0.9069565217391304

Iter 201 - train loss: [-0.93669], train acc: [0.96339], val acc: 0.9043478260869565

Iter 202 - train loss: [-0.94934], train acc: [0.96384], val acc: 0.9034782608695652

Iter 203 - train loss: [-0.9522], train acc: [0.96339], val acc: 0.9052173913043479

Iter 204 - train loss: [-0.99179], train acc: [0.96362], val acc: 0.9034782608695652

Iter 205 - train loss: [-0.97564], train acc: [0.9625

Iter 291 - train loss: [-0.9531], train acc: [0.97098], val acc: 0.9026086956521739

Iter 292 - train loss: [-0.95309], train acc: [0.97054], val acc: 0.9052173913043479

Iter 293 - train loss: [-0.97658], train acc: [0.97076], val acc: 0.9034782608695652

Iter 294 - train loss: [-0.96861], train acc: [0.97121], val acc: 0.9017391304347826

Iter 295 - train loss: [-0.98341], train acc: [0.97076], val acc: 0.9034782608695652

Iter 296 - train loss: [-0.96812], train acc: [0.97098], val acc: 0.9017391304347826

Iter 297 - train loss: [-0.96092], train acc: [0.97254], val acc: 0.9026086956521739

Iter 298 - train loss: [-0.97561], train acc: [0.97254], val acc: 0.9034782608695652

Iter 299 - train loss: [-0.95964], train acc: [0.97232], val acc: 0.9069565217391304

Iter 300 - train loss: [-0.98413], train acc: [0.97188], val acc: 0.9052173913043479

Iter 301 - train loss: [-0.96016], train acc: [0.97388], val acc: 0.9026086956521739

Iter 302 - train loss: [-0.95277], train acc: [0.9721],

...

Iter 579 - train loss: [-0.98265], train acc: [0.97924], val acc: 0.908695652173913

Iter 580 - train loss: [-0.9843], train acc: [0.97991], val acc: 0.9095652173913044

Iter 581 - train loss: [-0.96863], train acc: [0.9779], val acc: 0.908695652173913

Iter 582 - train loss: [-0.98435], train acc: [0.98013], val acc: 0.9069565217391304

Iter 583 - train loss: [-0.99215], train acc: [0.97857], val acc: 0.9121739130434783

Iter 584 - train loss: [-0.98703], train acc: [0.97969], val acc: 0.9147826086956522

Iter 585 - train loss: [-0.98425], train acc: [0.97924], val acc: 0.9113043478260869

Iter 586 - train loss: [-0.9682], train acc: [0.97857], val acc: 0.9060869565217391

Iter 587 - train loss: [-0.98334], train acc: [0.97879], val acc: 0.9078260869565218

Iter 588 - train loss: [-0.98434], train acc: [0.97991], val acc: 0.9095652173913044

Iter 589 - train loss: [-0.9842], train acc: [0.97879], val acc: 0.9078260869565218

Iter 590 - train loss: [-0.97194], train acc: [0.97835], val

Iter 675 - train loss: [-0.9841], train acc: [0.97857], val acc: 0.908695652173913

Iter 676 - train loss: [-0.97606], train acc: [0.97924], val acc: 0.9078260869565218

Iter 677 - train loss: [-0.99214], train acc: [0.97835], val acc: 0.9078260869565218

Iter 678 - train loss: [-0.97652], train acc: [0.97902], val acc: 0.9069565217391304

Iter 679 - train loss: [-0.99206], train acc: [0.98058], val acc: 0.9017391304347826

Iter 680 - train loss: [-0.96871], train acc: [0.97969], val acc: 0.9034782608695652

Iter 681 - train loss: [-0.98423], train acc: [0.97969], val acc: 0.9043478260869565

Iter 682 - train loss: [-0.98423], train acc: [0.98036], val acc: 0.9060869565217391

Iter 683 - train loss: [-0.96873], train acc: [0.97812], val acc: 0.9026086956521739

Iter 684 - train loss: [-0.99218], train acc: [0.97857], val acc: 0.9034782608695652

Iter 685 - train loss: [-0.95211], train acc: [0.97946], val acc: 0.9043478260869565

Iter 686 - train loss: [-0.97631], train acc: [0.97924],

Iter 771 - train loss: [-0.99448], train acc: [0.97857], val acc: 0.9104347826086957

Iter 772 - train loss: [-0.99216], train acc: [0.9779], val acc: 0.9095652173913044

Iter 773 - train loss: [-0.98465], train acc: [0.97857], val acc: 0.9060869565217391

Iter 774 - train loss: [-0.99128], train acc: [0.9779], val acc: 0.9060869565217391

Iter 775 - train loss: [-0.99213], train acc: [0.97723], val acc: 0.9052173913043479

Iter 776 - train loss: [-0.98424], train acc: [0.97946], val acc: 0.9095652173913044

Iter 777 - train loss: [-0.96863], train acc: [0.97946], val acc: 0.9104347826086957

Iter 778 - train loss: [-0.98433], train acc: [0.97969], val acc: 0.9078260869565218

Iter 779 - train loss: [-0.98423], train acc: [0.97879], val acc: 0.9060869565217391

Iter 780 - train loss: [-0.9764], train acc: [0.97969], val acc: 0.9069565217391304

Iter 781 - train loss: [-0.98431], train acc: [0.97924], val acc: 0.9104347826086957

Iter 782 - train loss: [-0.98436], train acc: [0.9808], v

Iter 867 - train loss: [-0.95308], train acc: [0.98013], val acc: 0.9069565217391304

Iter 868 - train loss: [-0.97235], train acc: [0.97902], val acc: 0.9060869565217391

Iter 869 - train loss: [-1.], train acc: [0.97946], val acc: 0.9052173913043479

Iter 870 - train loss: [-0.9763], train acc: [0.97946], val acc: 0.9095652173913044

Iter 871 - train loss: [-0.96857], train acc: [0.98013], val acc: 0.9078260869565218

Iter 872 - train loss: [-0.97693], train acc: [0.97946], val acc: 0.9078260869565218

Iter 873 - train loss: [-0.99999], train acc: [0.97857], val acc: 0.9052173913043479

Iter 874 - train loss: [-0.96865], train acc: [0.97879], val acc: 0.908695652173913

Iter 875 - train loss: [-0.98489], train acc: [0.9779], val acc: 0.9078260869565218

Iter 876 - train loss: [-0.99218], train acc: [0.97969], val acc: 0.9078260869565218

Iter 877 - train loss: [-0.9609], train acc: [0.97902], val acc: 0.9104347826086957

Iter 878 - train loss: [-0.98428], train acc: [0.97768], val ac

Iter 964 - train loss: [-0.97653], train acc: [0.97924], val acc: 0.9069565217391304

Iter 965 - train loss: [-0.99931], train acc: [0.98013], val acc: 0.9060869565217391

Iter 966 - train loss: [-0.98386], train acc: [0.97768], val acc: 0.9034782608695652

Iter 967 - train loss: [-0.99179], train acc: [0.97946], val acc: 0.9026086956521739

Iter 968 - train loss: [-0.99189], train acc: [0.97857], val acc: 0.9043478260869565

Iter 969 - train loss: [-0.97635], train acc: [0.97902], val acc: 0.9052173913043479

Iter 970 - train loss: [-0.98341], train acc: [0.97969], val acc: 0.9052173913043479

Iter 971 - train loss: [-0.96091], train acc: [0.97969], val acc: 0.9026086956521739

Iter 972 - train loss: [-0.9766], train acc: [0.97969], val acc: 0.9026086956521739

Iter 973 - train loss: [-0.98434], train acc: [0.97835], val acc: 0.9008695652173913

Iter 974 - train loss: [-0.96075], train acc: [0.98013], val acc: 0.9034782608695652

Iter 975 - train loss: [-0.98435], train acc: [0.97879]

In [ ]:
val_x = to_np(val_x)
modeldata.trn_ds.extend(val_x, val_y_pred)

In [ ]:
#torch.save(model.state_dict(), 'MaxBiRNN.pt')
class TextClassificationLearner(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)
        self.crit = F.nll_loss
        self.metrics = [accuracy]

In [ ]:
set_trainable(model, True)
basicmodel = SingleModel(model)
lr = 1e-4
optimizer = partial(torch.optim.Adam, lr=lr)
learner = TextClassificationLearner(modeldata, basicmodel, opt_fn=optimizer, tmp_name='test', models_name='model')

In [ ]:
#learning_rate = 1e-4
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#loss_fn = torch.nn.NLLLoss()

#fit(model, modeldata, epochs=10000, opt=optimizer, crit=loss_fn, metrics=[accuracy], callbacks=None)

In [ ]:
#model = model.cuda()

In [ ]:
learner.lr_find(start_lr=0.03, end_lr=1)
learner.sched.plot(n_skip=0, n_skip_end=1)

In [ ]:
learner.fit(lrs = [1e-3], n_cycle = 1, cycle_len=1000, cycle_mult=2)
#learner.lr_find(lr)
#learner.warm_up(lr)

In [ ]:
learner.fit(lrs = [1e-3], n_cycle = 1, cycle_len=10000)

In [ ]:
learner.sched.plot_lr()

In [ ]:
learning_rate = 1e-4
#optimizer = torch.optim.Adam(params, lr=learning_rate)
model.emb.weight.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate) # Work with sparse embedding, much faster, but worse result
loss_fn = torch.nn.NLLLoss()
i = 0
while i < 3000:
    model.train()
    for xs, ys in iter(modeldata.trn_dl):
        x_btch, y_btch = V(xs), V(ys)
        y_pred = model(x_btch)
        # Compute and print loss.
        loss = loss_fn(y_pred, y_btch)
        optimizer.zero_grad()
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()
        
    model.eval()
    val_x = V(modeldata.val_ds.xs)
    val_y = modeldata.val_ds.ys
    val_y_pred = to_np(model(val_x)).argmax(axis=1)
    #model.training = True
    acc = np.sum(val_y_pred == val_y)/(len(val_y))
    print(f'Iter {i} - val acc: {acc}')
    i = i + 1
    print(to_np(loss))

In [ ]:
100#w2v.dist('lammestek', 'kjøtt')
#w2v.dist('kylling', 'kjøtt')
#w2v.dist('beef', 'kjøtt')
#w2v.dist('beef', 'frukt')
#w2v.neighbors('lamme', 50)

#Predict
texts_test=['The burger Cheesy, original eller spicy. 4 x 80 g. 320 g. Pr kg 155,94', 
            'Coop svenske kjøttboller 400 g. Pr kg 69,00',
            'Barilla spaghetti 1 kg',
            'Barilla tagliatelle M/egg. 500 g. Pr kg 77,00',
            'Lofoten hjemmelaget fiskegrateng 400 g. Pr kg 195,50 600 Pr',
            'Barilla spaghetti fullkorn 1 kg',
            'Coop fra havet laksefilet 500 g. Pr kg 159,80',
            'Barilla saus classico 400 g. Pr kg 59,75',
            'Coop fra havet lettsaltet torskefilet M/skinn. 360 g. Pr kg 147,50',
            'Barilla pesto genovese 190 g. Pr kg 157,37']
xs = textproc.text2features(texts_test)
model.eval()
ys = model(V(xs))
ys = to_np(ys).argmax(axis=1)
labels = [modeldata.idx2label(y) for y in ys]

#Save model
#model.save_state_dict('MaxBiRNN.pt')

#torch.save(model.state_dict(), 'MaxBiRNN.pt')
#Load model

#new_model = MaxBiRNN(w2v, seqlen=modeldata.get_seqlen(), n_labels=modeldata.get_c(), hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
#new_model.load_state_dict(torch.load('MaxBiRNN.pt'))
#model = model.cuda()
#model

#Explain Prediction


In [ ]:
new_model.eval()
new_model.cuda()
ys = new_model(V(xs))
ys = to_np(ys).argmax(axis=1)
labels = [modeldata.idx2label(y) for y in ys]

In [ ]:
#torch.save(model.state_dict(), 'MaxBiRNN_10000_original.pt')


In [ ]:
from sklearn.model_selection import train_test_split
df = mattext.df
df = df.assign(text = lambda df: df.prodName + " " + df.prodDesc)[["catName", "text"]]
df = df.rename(columns = {"catName": "label", "text":"text"})
idx2label = df.label.unique()
label2idx = {idx2label[idx]: idx for idx in range(len(idx2label))}
n_labels = len(idx2label)

In [ ]:
#Split train and validation set
#TODO: Add price to dataset?
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'])

In [ ]:
w2v_tns = torch.FloatTensor(w2v.emb_mat)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# ~77%
class ConvTwiceEmbeddingNet(nn.Module):
    def __init__(self, w2v_tns, n_labels, p0=0.2, p1=0.2, p2=0.2, p3=0.2, p4=0.2):
        super().__init__()
        n_words, n_factors = w2v_tns.shape
        self.emb = nn.Embedding(n_words, n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = False
        self.dropout_emb = nn.Dropout(0.2)
        
        self.conv1 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size = 3)
        self.drop1 = nn.Dropout(p1)
        
        self.conv2 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=4, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size = 3)
        self.drop2 = nn.Dropout(p2)
        
        self.conv3 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=5, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size = 3)
        self.drop3 = nn.Dropout(p3)
        
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool1d(kernel_size = 3)
        
        self.flat = Flatten()
        
        self.lin1 = nn.Linear(1920, 128)
        self.relu4 = nn.ReLU()
        self.drop4 = nn.Dropout(p4)
        self.lin2 = nn.Linear(128, n_labels)
        
    def forward(self, input):
        x = self.emb(input)
        x = x.permute(0, 2, 1)
        x = self.dropout_emb(x)
        x1 = self.pool1(self.drop1(self.relu1(self.bn1(self.conv1(x)))))
        x2 = self.pool2(self.drop2(self.relu2(self.bn2(self.conv2(x)))))
        x3 = self.pool3(self.drop3(self.relu3(self.bn3(self.conv3(x)))))
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        #x = self.drop2(self.relu2(self.conv2(x)))
        x = torch.cat((x1, x2, x3), dim = 2)
        # print(x.shape)
        #IPython.core.debugger.Pdb().set_trace()
        x = self.flat(self.pool4(self.relu4(self.bn4(self.conv4(x)))))
        # print(x.shape)
        x = self.drop4(self.relu4(self.lin1(x)))
        x = self.lin2(x)
        return F.softmax(x)

# ~80.78 (p0=0.0, p1=0.5) Kim Yoon
# https://arxiv.org/pdf/1408.5882.pdf
class ConvEmbeddingNet(nn.Module):
    def __init__(self, w2v_tns, n_labels, p0=0.2, p1=0.2):
        super().__init__()
        n_words, n_factors = w2v_tns.shape
        self.emb = nn.Embedding(n_words, n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = True
        self.dropout_emb = nn.Dropout(p0)
        
        self.bn0 = nn.BatchNorm1d(n_factors)
        
        self.conv1 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size = 3)
        
        self.conv2 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=4)
        self.bn2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size = 3)
        
        self.conv3 = nn.Conv1d(in_channels=n_factors, out_channels=128, kernel_size=5)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size = 3)
        
        self.classifier = nn.Sequential(
                Flatten(),
                nn.Dropout(p1),
                nn.BatchNorm1d(384),
                #nn.Linear(384, 128),
                #nn.BatchNorm1d(128),
                #nn.ReLU(),
                #nn.Linear(128, n_labels)
                nn.Linear(384, n_labels)
                )
        
    def forward(self, input):
        x = self.emb(input)
        x = x.permute(0, 2, 1)
        x = self.dropout_emb(x)
        x1, _ = torch.max(self.relu1(self.bn1(self.conv1(x))), dim=2)
        x2, _ = torch.max(self.relu2(self.bn2(self.conv2(x))), dim=2)
        x3, _ = torch.max(self.relu3(self.bn3(self.conv3(x))), dim=2)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        #x = self.drop2(self.relu2(self.conv2(x)))
        x = torch.cat((x1, x2, x3), dim = 1)
        x = self.classifier(x)
        # print(x.shape)
        #IPython.core.debugger.Pdb().set_trace()
        #x = self.flat(self.pool4(self.relu4(self.bn4(self.conv4(x)))))
        #x = self.flat(self.drop3(x))
        # print(x.shape)
        #x = self.drop4(self.relu4(self.lin1(x)))
        #x = self.relu4(self.lin1(x))
        #x = self.lin2(x)
        return F.softmax(x)

#Concat min and max of word embedding, 80.6%
class MaxAvgEmbeddingNet(nn.Module): 
    def __init__(self, w2v_tns, n_labels, p1=0.2, p2=0.2, p3=0.2, init=None):
        super().__init__()
        n_words, n_factors = w2v_tns.shape
        self.emb = nn.Embedding(n_words, n_factors)
        self.emb.weight.data = w2v_tns
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(n_factors*2),
            nn.Dropout(p1),
            nn.Linear(n_factors*2, 2000),
            nn.BatchNorm1d(2000),
            nn.ReLU(True),
            nn.Dropout(p2),
            nn.Linear(2000, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(True),
            nn.Dropout(p3),
            nn.Linear(1000, n_labels)
        )
        
    def forward(self, input):
        x = self.emb(input)
        x = self.bn0(x)
        x = x.permute(0, 2, 1)
        #x = torch.mean(x, dim=2)
        x_max, _ = torch.max(x, dim=2)
        x_mean = torch.mean(x, dim=2)
        x = torch.cat((x_mean, x_max), dim = 1)
        x = self.classifier(x)
        return F.softmax(x)
    
#Old Max Embedding
class MaxEmbeddingNet(nn.Module): #66% at ~2000iters; p1=0.3; p2=0.1 | 74% train 3000iters fix embedding, then 2000iters with emb; lr=1e-4 3000iters
    def __init__(self, w2v_tns, n_labels, p1=0.2, p2=0.2):
        super().__init__()
        n_words, n_factors = w2v_tns.shape
        self.emb = nn.Embedding(n_words, n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = False
        self.drop1 = nn.Dropout(p1)
        self.lin1 = nn.Linear(n_factors, 1000)
        self.relu1 = nn.ReLU()
        self.lin2 = nn.Linear(1000, n_labels)
        self.drop2 = nn.Dropout(p2)
        
    def forward(self, input):
        x = self.emb(input)
        x = x.permute(0, 2, 1)
        #x = torch.mean(x, dim=2)
        x, _ = torch.max(x, dim=2)
        x = self.relu1(self.lin1(self.drop1(x)))
        x = self.lin2(self.drop2(x))
        return F.softmax(x)

# TODO: Sparse gradient for embedding
# TODO: Add L2, scale_grad_by_freq, padding_idx
# TODO: Fasttext lanuage model learning first then classify later. Get Opendata for training model with LSTM?
# TODO: Bidirectional LSTM using pretrained emb: https://www.kaggle.com/liliasimeonova/sentiment-analysis-with-bidirectional-lstm
# TODO: Write a blog post on doing Text classification / Sentiment Analysis... for rare language with limited labeled dataset
        # http://archive.ics.uci.edu/ml/datasets/Twitter+Data+set+for+Arabic+Sentiment+Analysis
        # https://github.com/duyvuleo/VNTC
        # "Translate to English" approach?
# Awesome Text Classification: https://github.com/fendouai/Awesome-Text-Classification
# Nice analyze paper: https://arxiv.org/pdf/1510.03820.pdf
# Example of Character-Level CNN and Very deep CNN in pytorch: https://github.com/ArdalanM/nlp-benchmarks
# TODO: Add LIME to explain text classifier: https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html
# TODO: Why it makes wrong predictions?
# TODO: Add EmbeddingDropout
# TODO: https://github.com/fastai/fastai/blob/master/fastai/lm_rnn.py
class EmbeddingNet(nn.Module): #80.51% p1=0.4; p2=0.3; p3=0.3; lr=1e-4 batch = 128
    def __init__(self, w2v_tns, n_labels, p1=0.2, p2=0.2, p3=0.1, init=None):
        super().__init__()
        n_words, n_factors = w2v_tns.shape
        self.emb = nn.Embedding(n_words, n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = True
        
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(n_factors),
            nn.Dropout(p1),
            nn.Linear(n_factors, 2000),
            nn.BatchNorm1d(2000),
            nn.ReLU(True),
            nn.Dropout(p2),
            nn.Linear(2000, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(True),
            nn.Dropout(p3),
            nn.Linear(1000, n_labels)
        )
        
        if init:
            init(self)
    def forward(self, input):
        x = self.emb(input)
        x = x.permute(0, 2, 1)
        #x = torch.mean(x, dim=2)
        x, _ = torch.max(x, dim=2)
        x = self.classifier(x)
        return F.softmax(x)


    
# 79.74%; 2445 iter hidden_size=192, n_layers=1, p0=0.2, p1=0.2, p2=0.2
# https://www.kaggle.com/liliasimeonova/sentiment-analysis-with-bidirectional-lstm
# https://github.com/keras-team/keras/blob/master/examples/imdb_bidirectional_lstm.py
class BiRNN(nn.Module):
    def __init__(self, w2v_tns, len_sent, n_labels, hidden_size, n_layers, p0=0.2, p1=0.2, p2=0.2):
        super().__init__()
        self.n_words, self.n_factors = w2v_tns.shape
        self.len_sent = len_sent
        self.emb = nn.Embedding(self.n_words, self.n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = False
        
        self.emb_bn = nn.BatchNorm1d(len_sent)
        self.emb_dropout = nn.Dropout(p0)
        
        self.hidden_size, self.n_labels, self.n_layers = hidden_size, n_labels, n_layers
        self.lstm = nn.LSTM(self.n_factors, hidden_size, n_layers, batch_first=True, bidirectional=True, dropout=p1)
        self.flat = Flatten()
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(hidden_size*2*self.len_sent),
            nn.Linear(hidden_size*2*self.len_sent, n_labels)
            #nn.BatchNorm1d(1000),
            #nn.ReLU(True),
            #nn.Dropout(p2),
            #nn.Linear(1000, n_labels)
        )        

    def forward(self, input):
        #Set initial state
        h0 = V(torch.zeros(self.n_layers*2, input.shape[0], self.hidden_size))
        c0 = V(torch.zeros(self.n_layers*2, input.shape[0], self.hidden_size))
        # Get embedding
        x = self.emb_bn(self.emb_dropout(self.emb(input)))
        # Forward propagate RNN
        x, _ = self.lstm(x, (h0, c0))
        #print(x.shape)
        #import pdb; pdb.set_trace()
        # Decode hidden state of last time step
        x = self.flat(x.contiguous())
        out = self.classifier(x)
        return F.softmax(out)

# Use only last hidden state of LSTM for classification
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py
# 80.38% iter 3713, 80.77 ~7000, hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2
class LastBiRNN(nn.Module):
    def  __init__(self, w2v_tns, len_sent, n_labels, hidden_size, n_layers, p0=0.2, p1=0.2, p2=0.2, p3=0.2):
        super().__init__()
        self.n_words, self.n_factors = w2v_tns.shape
        self.len_sent = len_sent
        self.emb = nn.Embedding(self.n_words, self.n_factors)
        self.emb.weight.data = w2v_tns
        self.emb.weight.requires_grad = True
        
        self.emb_bn = nn.BatchNorm1d(len_sent)
        self.emb_dropout = nn.Dropout(p0)
        
        self.hidden_size, self.n_labels, self.n_layers = hidden_size, n_labels, n_layers
        self.lstm = nn.LSTM(self.n_factors, hidden_size, n_layers, batch_first=True, bidirectional=True, dropout=p1)
        self.flat = Flatten()
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(hidden_size*2),
            nn.Dropout(p1),
            nn.Linear(hidden_size*2, 2000),
            nn.BatchNorm1d(2000),
            nn.ReLU(True),
            nn.Dropout(p2),
            nn.Linear(2000, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(True),
            nn.Dropout(p3),
            nn.Linear(1000, n_labels)
        )        

    def forward(self, input):
        #Set initial state
        h0 = V(torch.zeros(self.n_layers*2, input.shape[0], self.hidden_size))
        c0 = V(torch.zeros(self.n_layers*2, input.shape[0], self.hidden_size))
        # Get embedding
        x = self.emb_bn(self.emb_dropout(self.emb(input)))
        # Forward propagate RNN
        x, _ = self.lstm(x, (h0, c0))
        x    = x[:, -1, :] #Get last state
        x = self.flat(x.contiguous())
        out = self.classifier(x)
        return F.softmax(out)


# Google Translate and use English FastText w2v
def init(model):
    for m in model.modules():
        if isinstance(m, nn.Conv1d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
        elif isinstance(m, nn.BatchNorm1d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal(m.weight)
            m.bias.data.zero_()

In [ ]:
w2v_tns = T(w2v.emb_mat)
#model = ConvEmbeddingNet(w2v_tns, n_labels, p0=0.0, p1=0.5)
#model = MaxAvgEmbeddingNet(w2v_tns, n_labels, p1=0.3, p2=0.2, p3=0.2, init=None)
#model = EmbeddingNet(w2v_tns, modeldata.get_c(), p1=0.4, p2=0.3, p3=0.3)
#model = BiRNN(w2v_tns, len_sent=50, n_labels=n_labels, hidden_size=192, n_layers=1, p0=0.2, p1=0.2, p2=0.2)
model = MaxBiRNN(w2v_tns, len_sent=50, n_labels=modeldata.get_c(), hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
#model = LastBiRNN(w2v_tns, len_sent=50, n_labels=n_labels, hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
#model = MaxMinEmbeddingNet(w2v_tns, n_labels, p1=0.4, p2=0.3, p3=0.3, init=init)
#model = AttentionBiRNN(w2v_tns, len_sent=50, n_labels=n_labels, hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
model = model.cuda()
model

In [ ]:
learning_rate = 1e-4
#params = [param for name, param in model.named_parameters() if not name.startswith('emb')]
model.emb.weight.requires_grad = True
#optimizer = torch.optim.Adam(params, lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate) # Work with sparse embedding, much faster, but worse result
loss_fn = torch.nn.NLLLoss()

In [ ]:
#learning_rate = 1e-4
#for param_group in optimizer.param_groups:
#    param_group['lr'] = learning_rate

In [ ]:
trn_x = modeldata.trn_ds.xs
trn_y = modeldata.trn_ds.ys
tst_x = modeldata.val_ds.xs
tst_y = modeldata.val_ds.ys
x = V(trn_x)#.cpu()
y = V(np.asarray(trn_y))#.cpu()
#x = Variable(torch.LongTensor(trn_x))
#y = Variable(torch.LongTensor(np.asarray(trn_y)))
#val_x = Variable(torch.LongTensor(tst_x))
#val_y = Variable(torch.LongTensor(np.asarray(tst_y)))
val_x = V(tst_x)#.cpu()
val_y = V(np.asarray(tst_y))#.cpu()
#model = model.cuda()
model.training = True
bz = 128
start = 0
end = bz
i = 0
while i < 10000:
    if end > len(x):
        start = 0
        end = bz
        #new_order = torch.LongTensor(np.random.permutation(len(x)))
        new_order = T(np.random.permutation(len(x)))#.cpu()
        x = x[new_order]
        y = y[new_order]
        model.training = False
        val_y_pred = to_np(model(val_x)).argmax(axis=1)
        model.training = True
        acc = np.sum(val_y_pred == tst_y)/(len(val_y))
        print(f'Iter {i} - val acc: {acc}')
        i = i + 1
        print(to_np(loss))
        
    x_btch = x[start:end, ]
    y_btch = y[start:end, ]
    y_pred = model(x_btch)

    # Compute and print loss.
    loss = loss_fn(y_pred, y_btch)
    optimizer.zero_grad()
    # Backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # Calling the step function on an Optimizer makes an update to its parameters
    optimizer.step()
    
    end = end + bz
    start = start + bz

In [ ]:
# model.training = False
x = Variable(torch.LongTensor(trn_x))
y = Variable(torch.LongTensor(np.asarray(trn_y)))
y_max = to_np(model(x)).argmax(axis=1)
print(y_max[0:30])
print(trn_y[0:30])
acc = np.sum(y_max == trn_y)/(len(y_max))

In [ ]:
acc

In [ ]:
def create_emb(w2v):
    n_fact = w2v.n_factors()
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs50[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = np.random.normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = np.random.normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb


In [ ]:
class MixedInputModel(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                 y_range=None, use_bn=False):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)
        self.n_emb, self.n_cont=n_emb, n_cont
        
        szs = [n_emb+n_cont] + szs
        self.lins = nn.ModuleList([
            nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: kaiming_normal(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        kaiming_normal(self.outp.weight.data)

        self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        if self.y_range:
            x = F.sigmoid(x)
            x = x*(self.y_range[1] - self.y_range[0])
            x = x+self.y_range[0]
        return x
